In [74]:
from PIL import Image
from google import genai

In [75]:
import json

# Load your JSON
with open("/content/Train.json") as f:
    train_data = json.load(f)

with open("/content/Test.json") as f:
    test_data = json.load(f)

In [76]:
Titles_train = []
Outputs_train = []
for i in range(len(train_data['Few_Shots_Examples'])) :
  Titles_train.append(train_data['Few_Shots_Examples'][i]['input']['title'])
  Outputs_train.append(train_data['Few_Shots_Examples'][i]['output'])

In [77]:
Titles_test = []
Output_test = []
for i in range(len(test_data['test_data'])) :
  Titles_test.append(test_data['test_data'][i]['input']['title'])
  Output_test.append(test_data['test_data'][i]['output'])

In [78]:
Output_test

[{'summary': {'Ingredients': ['1 tablespoon vegetable oil',
    '2 cloves garlic, minced',
    '1 pound shredded cabbage',
    '1 tablespoon soy sauce',
    '1 tablespoon Chinese cooking wine (Shaoxing wine)'],
   'Instructions': ['Heat oil in a wok, add garlic, and cook for a few seconds until slightly browned',
    'Add shredded cabbage, stir to coat, cover, and cook for 1 minute',
    'Pour in soy sauce, stir for another minute, then add cooking wine and stir-fry until tender',
    'Serve hot and enjoy']}},
 {'summary': {'Ingredients': ['1 ¾ cups all-purpose flour, divided',
    '¾ cup warm water, divided',
    '1 tablespoon active dry yeast',
    '1 teaspoon white sugar',
    '2 tablespoons white sugar',
    '1 tablespoon vegetable oil',
    '¼ teaspoon salt',
    '½ teaspoon baking powder'],
   'Instructions': ['Mix 1/4 cup flour, 1/4 cup warm water, yeast, and 1 tsp sugar; let sit 30 mins',
    'Add remaining ingredients and knead to smooth dough; let rise until tripled',
    'Pu

In [79]:
SYSTEM_PROMPT = """
You are a world-class culinary expert and recipe generator with exceptional skills in analyzing both images and text.
Your role is to generate detailed, precise, and easy-to-follow cooking recipes based solely on a food item's image and its title.

When given an image and a noisy title, you will:

1. Observe the image carefully and identify visible ingredients, textures, and cooking styles.
2. Use the title to understand the cuisine type, dish category, and possible variations , dont get confused by it's vague name.It is a unique description for that item.
3. Think step-by-step about what ingredients would logically be included, considering common cooking methods for that dish.
4. Curate a relevant list of ingredients required to cook the item.
5. While generating the summary of the cooking recipe,break down the cooking process into clear, sequential instructions, ensuring clarity and precision.
6. Provide the final output recipe as two distinct sections: Ingredients and Instructions.
"""

In [80]:
# prompt = f""" {SYSTEM_PROMPT} \n\n
#         Below are some examples on what the input and output looks like,
#         you will be given the input image and title text {Titles_train}
#         and you have to generate the output in the format as shown below for each of the example : \n
#         {Outputs_train}
#         Strictly Follow the output format , No extra output needed
#          """

In [81]:
img_11 = Image.open("/content/Train_Images/pasta.jpg")
img_12 = Image.open("/content/Train_Images/burger.jpg")
img_13 = Image.open("/content/Train_Images/sandwich.jpg")
img_14 = Image.open("/content/Train_Images/Fish Tacos.jpg")
img_15 = Image.open("/content/Train_Images/Hot Honey Baked Beans.jpg")
img_16 = Image.open("/content/Train_Images/Marry Me Chickpeas.jpg")
img_17 = Image.open("/content/Train_Images/Bagels.jpg")
img_18 = Image.open("/content/Train_Images/Kheer (Rice Pudding).jpg")
img_19 = Image.open("/content/Train_Images/paneer tikka masala.jpg")
img_20 = Image.open("/content/Train_Images/Coffee Cake.jpg")
Images_train = [img_11,img_12,img_13,img_14,img_15,img_16,img_17,img_18,img_19,img_20]

In [82]:
prompt = f""" {SYSTEM_PROMPT} \n\n
        Below are some examples on what the input and output looks like,
        you will be given the input image {Images_train}and title text {Titles_train}
        and you have to generate the output in the format as shown below for each of the example : \n
        {Outputs_train}
        Strictly Follow the output format , No extra output needed
         """

In [83]:
Titles_test

['Fried Cabbage',
 'Fluffy Buns',
 'Noodly Stir Fry',
 'Breakfast Wrap',
 'Chicken Bits']

In [84]:
img_1 = Image.open("/content/Cabbage Stir Fry.jpg")
img_2 = Image.open("/content/steamed buns.jpg")
img_3 = Image.open("/content/Fried Noodles.jpg")
img_4 = Image.open("/content/burrito.jpg")
img_5 = Image.open("/content/chicken bites.jpg")
Images_test = [img_1,img_2,img_3,img_4,img_5]

In [85]:
client = genai.Client(api_key="GEMINI_API_KEY") # Kripya Apni API daale
responses = []

for i in range(len(Titles_test)) :
  title = Titles_test[i]

  image = Images_test[i]
  final_prompt = prompt + f" \n For the given Input Image and Title : {title} , Give the Recipe Summary Output with ingredients and instructions in about 3-4 steps"
  response = client.models.generate_content(
      model="gemini-2.0-flash",
      contents=[image, final_prompt]
  )
  responses.append(response.text)
responses


["```json\n[\n  {\n    'summary': {\n      'Ingredients': [\n        '1 medium head of cabbage, shredded',\n        '1/4 cup bacon grease or vegetable oil',\n        '1/2 medium onion, sliced',\n        '2 cloves garlic, minced',\n        '1/4 cup chicken broth or water',\n        '2 tablespoons soy sauce',\n        '1 tablespoon rice vinegar',\n        '1 teaspoon sugar',\n        'Salt and pepper to taste'\n      ],\n      'Instructions': [\n        'Heat bacon grease or oil in a large skillet over medium-high heat. Add onion and cook until softened.',\n        'Add garlic and shredded cabbage. Stir-fry until cabbage is tender-crisp and slightly browned.',\n        'Pour in chicken broth or water, soy sauce, rice vinegar, and sugar. Season with salt and pepper. Cook until liquid is absorbed.',\n        'Serve hot as a side dish.'\n      ]\n    }\n  }\n]\n```",
 '```json\n[\n  {"summary": {"Ingredients": ["2 cups all-purpose flour", "1 cup warm milk", "2 tablespoons sugar", "1 teaspoo

In [86]:
import json
import re

# Your raw data
raw_data = responses
cleaned_data = []

for item in raw_data:
    # Step 1: Remove Markdown syntax
    cleaned = re.sub(r"```json\n|```", "", item.strip())

    # Step 2: Replace single quotes with double quotes (basic fix)
    cleaned = cleaned.replace("'", '"')

    # Step 3: Ensure it’s wrapped in a list (if not already)
    if not cleaned.strip().startswith("["):
        cleaned = f"[{cleaned}]"

    try:
        # Step 4: Load to Python object
        parsed = json.loads(cleaned)
        cleaned_data.extend(parsed)  # combine all into one list
    except json.JSONDecodeError as e:
        print(f"Failed to parse:\n{cleaned[:100]}...\nError: {e}\n")

# Step 5: Save final combined JSON
with open("generated_output.json", "w") as f:
    json.dump(cleaned_data, f, indent=4)

In [87]:
Generated_test = cleaned_data

In [36]:
!pip install rouge-score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=9ba0b83d47c5d56c437e10acb859db3e9db94b7e770689c375416cc7255d657b
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge-score


In [88]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer

smoothie = SmoothingFunction().method4
rouge = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

bleu_scores = []
rouge1_scores = []
rouge2_scores = []
rougeL_scores = []

for pred, actual in zip(Generated_test, Output_test):
    pred_text = " ".join(pred["summary"]["Ingredients"] + pred["summary"]["Instructions"])
    actual_text = " ".join(actual["summary"]["Ingredients"] + actual["summary"]["Instructions"])

    # BLEU
    reference = [actual_text.split()]
    hypothesis = pred_text.split()
    bleu = sentence_bleu(reference, hypothesis, smoothing_function=smoothie)
    bleu_scores.append(bleu)

    # ROUGE
    rouge_result = rouge.score(actual_text, pred_text)
    rouge1_scores.append(rouge_result['rouge1'].fmeasure)
    rouge2_scores.append(rouge_result['rouge2'].fmeasure)
    rougeL_scores.append(rouge_result['rougeL'].fmeasure)

# Average scores
avg_bleu = sum(bleu_scores) / len(bleu_scores)
avg_rouge1 = sum(rouge1_scores) / len(rouge1_scores)
avg_rouge2 = sum(rouge2_scores) / len(rouge2_scores)
avg_rougeL = sum(rougeL_scores) / len(rougeL_scores)

print(f"BLEU Score: {avg_bleu:.4f}")
print(f"ROUGE-1 F1: {avg_rouge1:.4f}")
print(f"ROUGE-2 F1: {avg_rouge2:.4f}")
print(f"ROUGE-L F1: {avg_rougeL:.4f}")


BLEU Score: 0.0461
ROUGE-1 F1: 0.4650
ROUGE-2 F1: 0.1612
ROUGE-L F1: 0.2910
